In [66]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.utils import plot_model, to_categorical
from keras.callbacks import TensorBoard

from keras.datasets import cifar10

import tensorflow as tf
from tensorflow import keras

# ヘルパーライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)


2.1.0


# データの読み込み
http://meipuru-344.hatenablog.com/entry/2017/11/27/205448   より

中身の説明

1 1 1 1 1 1 1 1 1 1 2 1 2 2 2 1 1 1 1 2 1 1 2 1 1 2 1 1 1 1 2 1 1 2 1 1 1 1 2 1 1 2 1 2 1 2 2 1 1 1 1 1 2 2 2 1 1 1 1 2 2 2 0 0 7 8 W 2

こんな感じのデータが大量に入っていると思います

八文字ごとに改行します

1 1 1 1 1 1 1 1
1 1 2 1 2 2 2 1
1 1 1 2 1 1 2 1
1 2 1 1 1 1 2 1
1 2 1 1 1 1 2 1
1 2 1 2 1 2 2 1
1 1 1 1 2 2 2 1
1 1 1 2 2 2 0 0

7 8 W 2

するとこのようになると思います 上の8 x 8の数字は現在の盤面です また黒が2、白が1です0は空白を表しています

この盤面に対して白が打った次の手が7 8(一番左上のマス目は(0,0)です)となります

最後の2はあと何個空白スペースがあるかを表しています

自分は勝った方のデータのみを用いて学習を行ったため

テキストに入っているデータは全て勝利した人の手が書かれています

またこの棋譜は

https://www.skatgame.net/mburo/ggs/game-archive/Othello/

を元に作成しました、オンラインオセロ対局のデータのようです


In [54]:
def transform(string,W):
    flat = string.replace("\n", "").split(" ")
    state = np.array([int(flat[j]) for j in range(64)]).reshape(8,8)
    if(W==1):
      state=(3-state)%3
    action = (int(flat[65])-1)*8 + int(flat[64])-1
    return state, action

In [55]:
with open("data\OthelloTeacher.txt") as f:
    data=f.readlines()

In [56]:
B_list=[];
W_list=[];
print(len(data))
for line in data:
    if "B" in line:
        B_list.append(line)
    else:
        W_list.append(line)
print(len(B_list),len(W_list))

148718
105074 43644


In [70]:
state_list=np.zeros([len(data),8,8])
action_list=np.zeros([len(data)])
for index,line in enumerate(data):
    if "B" in line:
        state,action=transform(line,0)
    else:
        state,action=transform(line,1)
    state_list[index]=state
    action_list[index]=action

In [72]:
state_list_1=np.where(state_list==1,1,0)
state_list_2=np.where(state_list==2,1,0)


In [74]:
display(state_list_1[1],state_list_2[1],action_list[1])

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

34.0

In [77]:
train=np.array([state_list_1,state_list_2])
display(train)

array([[[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 1, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 1, 0, 0],
         ...,
         [0, 0, 1, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        ...,

        [[0, 1, 1, ..., 1, 0, 1],
         [1, 0, 1, ..., 1, 0, 1],
         [1, 1, 1, ..., 0, 0, 1],
         ...,
         [1, 1, 0, ..., 0, 0, 1],
         [0, 0, 0, ..., 0, 0, 0],
         [1, 1, 1, ..., 1, 1, 1]],

        [[0, 1, 1, ..., 1, 0, 1],
         [1, 0, 1, ..., 0, 0, 1],
         [1, 1, 1, ..., 0, 0, 1],
    

In [80]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(8, 8, 2)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='softmax')
])

In [81]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [82]:
model.fit(train, action_list, epochs=5)

ValueError: Error when checking input: expected flatten_2_input to have shape (8, 8, 2) but got array with shape (148718, 8, 8)